In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
import sentencepiece
import transformers
import pandas as pd
import numpy as np
import gdown
import os.path
import torch
from transformers import T5ForConditionalGeneration , T5Tokenizer

print(transformers.__version__)
# print(sentencepiece.__version__)

4.11.3


In [ ]:
from datasets import load_dataset
dataset = load_dataset("paws-x","en")

Reusing dataset pawsx (/root/.cache/huggingface/datasets/pawsx/en/1.1.0/a5033b43902a02a4ba2ee469c1dd22af3e6a4a247ac47fa1af9835d0e734e2af)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

In [ ]:
train_df = pd.DataFrame(data=dataset['train'][:])
validation_df = pd.DataFrame(data=dataset['validation'][:])
test_df = pd.DataFrame(data=dataset['test'][:])
display(train_df.head())
display(validation_df.head())
display(test_df.head())

,id,sentence1,sentence2,label
0,1,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,3,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1


,id,sentence1,sentence2,label
0,4,From the merger of the Four Rivers Council and...,Shawnee Trails Council was formed from the mer...,1
1,6,Kathy and her husband Pete Beale ( Peter Dean ...,Kathy and her husband Peter Dean ( Pete Beale ...,1
2,7,Timora diarhoda is a species of moth of the No...,Diarhoda is a kind of moth of the Noctuidae fa...,1
3,9,Joe R. Campa Jr. is a former sailor of the Uni...,Joe R. Campa Jr. is a former U.S. Navy Matrose...,1
4,13,"Cook Pond , also known as the South Watuppa Po...","Cook Pond , also formerly known as Laurel Lake...",0


,id,sentence1,sentence2,label
0,10,The exception was between late 2005 and 2009 w...,"The exception was between late 2005 and 2009 ,...",1
1,12,The Tabaci River is a tributary of the River L...,The Leurda River is a tributary of the River T...,0
2,20,He played with the A-level Kane County Cougars...,He played in 1993 with the A - Level Portland ...,0
3,26,"Winarsky is a member of the IEEE , Phi Beta Ka...","Winarsky is a member of ACM , the IEEE , the P...",1
4,27,In 1938 he became the government anthropologis...,In 1938 he became the Government Anthropologis...,0


In [ ]:
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    print("Yes")
    torch.cuda.manual_seed_all(seed)

set_seed(42)

Yes


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-large')
tokenizer = T5Tokenizer.from_pretrained('t5-large')

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='t5-large', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>'

In [ ]:
def get_similarity(s):
  input_ids = tokenizer.encode(str(s) , return_tensors='pt')
  outputs = model.generate(input_ids=input_ids)
  sim_yd = tokenizer.decode(outputs[0])
  return sim_yd

In [ ]:
test_df['Similarity_Prep'] = "stsb sentence1:" + test_df['sentence1'] + " sentence2:" + test_df['sentence2']
print(test_df.shape)
test_df

(2000, 5)


,id,sentence1,sentence2,label,Similarity_Prep
0,10,The exception was between late 2005 and 2009 w...,"The exception was between late 2005 and 2009 ,...",1,stsb sentence1:The exception was between late ...
1,12,The Tabaci River is a tributary of the River L...,The Leurda River is a tributary of the River T...,0,stsb sentence1:The Tabaci River is a tributary...
2,20,He played with the A-level Kane County Cougars...,He played in 1993 with the A - Level Portland ...,0,stsb sentence1:He played with the A-level Kane...
3,26,"Winarsky is a member of the IEEE , Phi Beta Ka...","Winarsky is a member of ACM , the IEEE , the P...",1,stsb sentence1:Winarsky is a member of the IEE...
4,27,In 1938 he became the government anthropologis...,In 1938 he became the Government Anthropologis...,0,stsb sentence1:In 1938 he became the governmen...
...,...,...,...,...,...
1995,7984,Bas Basie Land is a studio album by Count Basi...,Land Basie Land is a studio album by Billy Bye...,0,stsb sentence1:Bas Basie Land is a studio albu...
1996,7992,Their mission is to guard various space outpos...,Their mission is to guard various outposts fro...,0,stsb sentence1:Their mission is to guard vario...
1997,7993,"On his mother , Elizabeth 's side were Sir Wil...",On his mother 's side were Elizabeth William M...,0,"stsb sentence1:On his mother , Elizabeth 's si..."
1998,7999,In 2014 the site launched iOS and Android appl...,In 2014 launched the site iOS and Android - ap...,1,stsb sentence1:In 2014 the site launched iOS a...


In [ ]:
# from tqdm import tqdm
from tqdm.auto import tqdm

In [ ]:
test_df['Sim_values'] = [get_similarity(x)[6:-4] for x in tqdm(test_df['Similarity_Prep'])]
test_df['Sim_values'] = pd.to_numeric(test_df['Sim_values'] , errors='coerce')
test_df['Predicted'] = 0

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
for z in range(len(test_df)):
  if test_df['Sim_values'][z] >= 4.5:
    test_df['Predicted'][z] = 1

display(test_df)
# test = "stsb sentence1:He likes bitches . sentence2:Bitches like him"
# # test = list(test_df['Similarity_Prep'])[2]
# print(test)
# s = (get_similarity(test))[6:-4]
# print(pd.to_numeric(s))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,sentence1,sentence2,label,Similarity_Prep,Sim_values,Predicted
0,10,The exception was between late 2005 and 2009 w...,"The exception was between late 2005 and 2009 ,...",1,stsb sentence1:The exception was between late ...,5.0,1
1,12,The Tabaci River is a tributary of the River L...,The Leurda River is a tributary of the River T...,0,stsb sentence1:The Tabaci River is a tributary...,3.8,0
2,20,He played with the A-level Kane County Cougars...,He played in 1993 with the A - Level Portland ...,0,stsb sentence1:He played with the A-level Kane...,5.0,1
3,26,"Winarsky is a member of the IEEE , Phi Beta Ka...","Winarsky is a member of ACM , the IEEE , the P...",1,stsb sentence1:Winarsky is a member of the IEE...,5.0,1
4,27,In 1938 he became the government anthropologis...,In 1938 he became the Government Anthropologis...,0,stsb sentence1:In 1938 he became the governmen...,5.0,1
...,...,...,...,...,...,...,...
1995,7984,Bas Basie Land is a studio album by Count Basi...,Land Basie Land is a studio album by Billy Bye...,0,stsb sentence1:Bas Basie Land is a studio albu...,5.0,1
1996,7992,Their mission is to guard various space outpos...,Their mission is to guard various outposts fro...,0,stsb sentence1:Their mission is to guard vario...,4.6,1
1997,7993,"On his mother , Elizabeth 's side were Sir Wil...",On his mother 's side were Elizabeth William M...,0,"stsb sentence1:On his mother , Elizabeth 's si...",4.8,1
1998,7999,In 2014 the site launched iOS and Android appl...,In 2014 launched the site iOS and Android - ap...,1,stsb sentence1:In 2014 the site launched iOS a...,5.0,1


In [ ]:
cnt = 0
total = len(test_df)
for _,row in test_df.iterrows():
  if row['Predicted'] == row['label']:
    cnt += 1

print("ACCURACY" , cnt / total)

ACCURACY 0.5585


In [ ]:
from transformers import T5Model

In [ ]:
modelT5 = T5Model.from_pretrained('t5-large')

In [ ]:
test1 = test_df['sentence1'][0]
test2 = test_df['sentence2'][0]
print(test1)
print(test2)

The exception was between late 2005 and 2009 when he played in Sweden with Carlstad United BK , Serbia with FK Borac Čačak and Russian FC Terek Grozny .
The exception was between late 2005 and 2009 , when he played in Sweden with Carlstad United BK , Serbia with FK Borac Čačak and the Russian FC Terek Grozny .


In [ ]:
enc = tokenizer(test1 , return_tensors="pt")
out = modelT5.encoder(input_ids=enc['input_ids'] , attention_mask=enc['attention_mask'] , return_dict=True)

In [ ]:
pooled_sentence = out.last_hidden_state

In [ ]:
enc1 = torch.mean(pooled_sentence, dim=1)

In [ ]:
print(enc1.shape)

torch.Size([1, 1024])


In [ ]:
enc = tokenizer(test2 , return_tensors="pt")
out = modelT5.encoder(input_ids=enc['input_ids'] , attention_mask=enc['attention_mask'] , return_dict=True)

In [ ]:
pooled_sentence = out.last_hidden_state

In [ ]:
enc2 = torch.mean(pooled_sentence, dim=1)

In [ ]:
emb1 = enc1.detach().numpy()
emb2 = enc2.detach().numpy()

In [ ]:
emb1 = np.swapaxes(emb1[0].reshape(-1,1) , 0 , 1)
emb2 = np.swapaxes(emb2[0].reshape(-1,1) , 0 , 1)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
print(cosine_similarity(emb1 , emb2))

[[0.98642427]]


In [ ]:
def get_cosine_score(s1 , s2):
  # print(s1,s2)
  enc1 = tokenizer(s1 , return_tensors="pt")
  out1 = modelT5.encoder(input_ids=enc1['input_ids'] , attention_mask=enc1['attention_mask'] , return_dict=True)
  pooled_sent = out1.last_hidden_state
  tensor_emb1 = torch.mean(pooled_sent , dim=1)
  emb1 = tensor_emb1.detach().numpy()
  emb1 = np.swapaxes(emb1[0].reshape(-1,1) , 0 , 1)

  enc2 = tokenizer(s2 , return_tensors="pt")
  out2 = modelT5.encoder(input_ids=enc2['input_ids'] , attention_mask=enc2['attention_mask'] , return_dict=True)
  pooled_sent = out2.last_hidden_state
  tensor_emb2 = torch.mean(pooled_sent , dim=1)
  emb2 = tensor_emb2.detach().numpy()
  emb2 = np.swapaxes(emb2[0].reshape(-1,1) , 0 , 1)

  cs = cosine_similarity(emb1 , emb2)
  return cs

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# test_df['Cos_values'] = [get_cosine_score(x) for x in tqdm(test_df['Similarity_Prep'])]
# test_df['Sim_values'] = pd.to_numeric(test_df['Sim_values'] , errors='coerce')
# test_df['Predicted'] = 0
test_df['Cos_values'] = 0.0
for z in tqdm(range(len(test_df))):
  # print(z)
  cv = get_cosine_score(test_df['sentence1'][z] , test_df['sentence2'][z])
  test_df['Cos_values'][z] = cv

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
test_df

,id,sentence1,sentence2,label,Similarity_Prep,Sim_values,Predicted,Cos_values
0,10,The exception was between late 2005 and 2009 w...,"The exception was between late 2005 and 2009 ,...",1,stsb sentence1:The exception was between late ...,5.0,1,0.992426
1,12,The Tabaci River is a tributary of the River L...,The Leurda River is a tributary of the River T...,0,stsb sentence1:The Tabaci River is a tributary...,3.8,0,0.986424
2,20,He played with the A-level Kane County Cougars...,He played in 1993 with the A - Level Portland ...,0,stsb sentence1:He played with the A-level Kane...,5.0,1,0.952573
3,26,"Winarsky is a member of the IEEE , Phi Beta Ka...","Winarsky is a member of ACM , the IEEE , the P...",1,stsb sentence1:Winarsky is a member of the IEE...,5.0,1,0.976083
4,27,In 1938 he became the government anthropologis...,In 1938 he became the Government Anthropologis...,0,stsb sentence1:In 1938 he became the governmen...,5.0,1,0.954327
...,...,...,...,...,...,...,...,...
1995,7984,Bas Basie Land is a studio album by Count Basi...,Land Basie Land is a studio album by Billy Bye...,0,stsb sentence1:Bas Basie Land is a studio albu...,5.0,1,0.960723
1996,7992,Their mission is to guard various space outpos...,Their mission is to guard various outposts fro...,0,stsb sentence1:Their mission is to guard vario...,4.6,1,0.960617
1997,7993,"On his mother , Elizabeth 's side were Sir Wil...",On his mother 's side were Elizabeth William M...,0,"stsb sentence1:On his mother , Elizabeth 's si...",4.8,1,0.978066
1998,7999,In 2014 the site launched iOS and Android appl...,In 2014 launched the site iOS and Android - ap...,1,stsb sentence1:In 2014 the site launched iOS a...,5.0,1,0.852010


In [ ]:
test_df['Predicted_cos'] = 0
for z in range(len(test_df)):
  if test_df['Cos_values'][z] >= 0.99:
    test_df['Predicted_cos'][z] = 1

display(test_df)

,id,sentence1,sentence2,label,Similarity_Prep,Sim_values,Predicted,Cos_values,Predicted_cos
0,10,The exception was between late 2005 and 2009 w...,"The exception was between late 2005 and 2009 ,...",1,stsb sentence1:The exception was between late ...,5.0,1,0.992426,1
1,12,The Tabaci River is a tributary of the River L...,The Leurda River is a tributary of the River T...,0,stsb sentence1:The Tabaci River is a tributary...,3.8,0,0.986424,0
2,20,He played with the A-level Kane County Cougars...,He played in 1993 with the A - Level Portland ...,0,stsb sentence1:He played with the A-level Kane...,5.0,1,0.952573,0
3,26,"Winarsky is a member of the IEEE , Phi Beta Ka...","Winarsky is a member of ACM , the IEEE , the P...",1,stsb sentence1:Winarsky is a member of the IEE...,5.0,1,0.976083,0
4,27,In 1938 he became the government anthropologis...,In 1938 he became the Government Anthropologis...,0,stsb sentence1:In 1938 he became the governmen...,5.0,1,0.954327,0
...,...,...,...,...,...,...,...,...,...
1995,7984,Bas Basie Land is a studio album by Count Basi...,Land Basie Land is a studio album by Billy Bye...,0,stsb sentence1:Bas Basie Land is a studio albu...,5.0,1,0.960723,0
1996,7992,Their mission is to guard various space outpos...,Their mission is to guard various outposts fro...,0,stsb sentence1:Their mission is to guard vario...,4.6,1,0.960617,0
1997,7993,"On his mother , Elizabeth 's side were Sir Wil...",On his mother 's side were Elizabeth William M...,0,"stsb sentence1:On his mother , Elizabeth 's si...",4.8,1,0.978066,0
1998,7999,In 2014 the site launched iOS and Android appl...,In 2014 launched the site iOS and Android - ap...,1,stsb sentence1:In 2014 the site launched iOS a...,5.0,1,0.852010,0


In [ ]:
cnt = 0
total = len(test_df)
for _,row in test_df.iterrows():
  if row['Predicted_cos'] == row['label']:
    cnt += 1

print("ACCURACY" , cnt / total)

ACCURACY 0.5695
